In [ ]:
# ====== BASIC MODEL ====== #
# - Static model where the membrane potential is updated based on the input current

import numpy as np
import os
import matplotlib.pyplot as plt

# ====== MODEL PARAMETERS ====== #
#   - a, b, c, and d are neuron parameters (Izhikevich model)
#   - v_th is the threshold voltage
#   - Cm is the membrane capacitance
#   - K_x is a gain factor
#   - I is the initial current
# ============================== #
def InitializeNeuron(neuron_type, neuron_response):
    if neuron_response == 0:
        return {
            'a': 0.025,
            'b': 0.2,
            'c': -65,
            'd': 6,
            'v_th': 0.03,
            'Cm': 1,
            'K_one': 0.7,
            'K_two': 3,
            'neuron_type': neuron_type
        }
    else:
        return {
            'a': 0.02,
            'b': 0.25,
            'c': -50,
            'd': 1.5,
            'v_th': 0.03,
            'Cm': 1,
            'K_one': 0.7,
            'K_two': 3,
            'neuron_type': neuron_type
        }

# ====== MODEL EQUATIONS ====== #
#   - dv and du are the differential equations for the membrane potential and recovery variable
#   - Use Euler's method to update the state of the neuron
# ============================= #
def UpdateNeuronState(parameters, v, u, t, dt, I):
    a, b, c, d, v_th, Cm, K_one, K_two, neuron_type = parameters.values()

    for i in range(4):
        if i < 2:
            I[i * 20000:(i + 1) * 20000] = i * 10
        else:
            I[i * 20000:(i + 1) * 20000] = (i - 4) * -10

    if neuron_type == 0:
        dv = (0.04 * v[t]**2) + (5 * v[t]) + 140 - u[t] + (K_one / Cm) * I[t]
        du = a * (b * v[t] - u[t])

        if v[t] > v_th:
            v[t] = c
            u[t] += d

        u[t+1] = u[t] + dt * du
        v[t+1] = v[t] + dt * dv

    else:
        dv = (0.04 * v[t]**2) + (5 * v[t]) + 140 - u[t]
        du = a * (b * v[t] - u[t])

        if v[t] > v_th:
            v[t] = c
            u[t] += d

        u[t+1] = u[t] + dt * du
        v[t+1] = v[t] + dt * dv + (K_two / Cm) * (I[t + 1] - I[t])

# ====== PLOT RESULTS ====== #
#   - Membrane Potential vs Time plots the membrane potential over 1000 ms
#   - Add plot for recovery variable
#   - Add plot for firing rate vs current
# ========================== #
def PlotNeuronResults(time, v, I):
    output_folder = "Output_Data"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    plt.figure(figsize=(14, 4))
    plt.plot(time, v)
    plt.title('Membrane Potential vs Time')
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane Potential (mV)')
    plt.savefig(os.path.join(output_folder, 'Basic_Model_Output.png'))
    plt.show()

    plt.figure(figsize=(14, 4))
    plt.plot(time, I)
    plt.title('Current vs Time')
    plt.xlabel('Time (ms)')
    plt.ylabel('Current (nA)')
    plt.savefig(os.path.join(output_folder, 'Basic_Model_Current_Input.png'))
    plt.show()

# ====== SIMULATE NEURON ====== #
#   - Given the type of neuron and parameters, simulate the neuron over 1000 ms
#   - Initialize the membrane potential and recovery variable at t = 0
#   - Return the time and membrane potential
# ============================ #
def NeuronSimulate(parameters, dt, total_time, I):
    num_steps = int(total_time / dt)

    v = np.zeros(num_steps)
    u = np.zeros(num_steps)
    time = np.linspace(0, total_time, num_steps)

    v[0] = -65
    u[0] = 0

    for t in range(num_steps - 1):
        UpdateNeuronState(parameters, v, u, t, dt, I)

    return time, v

# ====== MAIN ====== #
#   - Contains all functions of the model
#   - Initialize the neuron type (Spiking = 0, Bursting = 1)
#   - Simulate the neuron
#   - Plot the results
# ================== #
def main():
    # Neuron type (SA-1 = 0, FA-1 = 1)
    neuron_type = 0
    # Neuron response (Spiking = 0, Bursting = 1)
    neuron_response = 0

    I = np.zeros(100000)

    parameters = InitializeNeuron(neuron_type, neuron_response)
    dt = 0.01
    total_time = 1000
    time, v = NeuronSimulate(parameters, dt, total_time, I)
    PlotNeuronResults(time, v, I)

if __name__ == "__main__":
    main()



In [ ]:
# ====== STATIC MODEL ====== #
#  - This version applies the model statically over an array that simulates a surface texture
#  - Imagine a finger pressing on a surface, but no scanning movement

import numpy as np
import os
import matplotlib.pyplot as plt

# ====== MODEL PARAMETERS ====== #
#   - a, b, c, and d are neuron parameters (Izhikevich model)
#   - v_th is the threshold voltage
#   - Cm is the membrane capacitance
#   - K_x is a gain factor
#   - I is the initial current
# ============================== #
def InitializeNeuron(neuron_type, neuron_response):
    if neuron_response == 0:
        return {
            'a': 0.025,
            'b': 0.2,
            'c': -65,
            'd': 6,
            'v_th': 0.03,
            'Cm': 1,
            'K_one': 0.7,
            'K_two': 3,
            'neuron_type': neuron_type
        }
    else:
        return {
            'a': 0.02,
            'b': 0.25,
            'c': -50,
            'd': 1.5,
            'v_th': 0.03,
            'Cm': 1,
            'K_one': 0.7,
            'K_two': 3,
            'neuron_type': neuron_type
        }

# ====== MODEL EQUATIONS ====== #
#   - dv and du are the differential equations for the membrane potential and recovery variable
#   - Use Euler's method to update the state of the neuron
# ============================= #
def UpdateNeuronState(parameters, v, u, t, dt, I, spike_count):
    a, b, c, d, v_th, Cm, K_one, K_two, neuron_type = parameters.values()

    if neuron_type == 0:
        if(I[t] == 0):
            spike_count = 0

        else:
            dv = (0.04 * v[t]**2) + (5 * v[t]) + 140 - u[t] + (K_one / Cm) * I[t]
            du = a * (b * v[t] - u[t])

            if v[t] > v_th:
                v[t] = c
                u[t] += d
                spike_count += 1

            u[t+1] = u[t] + dt * du
            v[t+1] = v[t] + dt * dv

    else:
        dv = (0.04 * v[t]**2) + (5 * v[t]) + 140 - u[t]
        du = a * (b * v[t] - u[t])

        if v[t] > v_th:
            v[t] = c
            u[t] += d
            spike_count += 1


        u[t+1] = u[t] + dt * du
        v[t+1] = v[t] + dt * dv + (K_two / Cm) * (I[t + 1] - I[t])

    return spike_count

# ====== PLOT RESULTS ====== #
#   - Membrane Potential vs Time plots the membrane potential over 1000 ms
#   - Add plot for recovery variable
#   - Add plot for firing rate vs current
# ========================== #
def PlotNeuronResults(time, v, I):
    output_folder = "Output_Data"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    plt.figure(figsize=(14, 4))
    plt.plot(time, v)
    plt.title('Membrane Potential vs Time')
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane Potential (mV)')
    plt.savefig(os.path.join(output_folder, 'Basic_Model_Output.png'))
    plt.show()

    plt.figure(figsize=(14, 4))
    plt.plot(time, I)
    plt.title('Current vs Time')
    plt.xlabel('Time (ms)')
    plt.ylabel('Current (nA)')
    plt.savefig(os.path.join(output_folder, 'Basic_Model_Current_Input.png'))
    plt.show()


# ====== SIMULATE NEURON ====== #
#   - Given the type of neuron and parameters, simulate the neuron over 1000 ms
#   - Initialize the membrane potential and recovery variable at t = 0
#   - Return the time and membrane potential
# ============================ #
def NeuronSimulate(parameters, dt, total_time, I):
    num_steps = int(total_time / dt)

    v = np.zeros(num_steps)
    u = np.zeros(num_steps)
    time = np.linspace(0, total_time, num_steps)

    v[0] = -65
    u[0] = 0

    spike_count = 0  # Counter for spikes

    for t in range(num_steps - 1):
        spike_count = UpdateNeuronState(parameters, v, u, t, dt, I, spike_count)

    return time, v, spike_count

# ====== MAIN ====== #
#   - Contains all functions of the model
#   - Initialize the neuron type (Spiking = 0, Bursting = 1)
#   - Simulate the neuron
#   - Plot the results
# ================== #
def main():
    # Neuron type (SA-1 = 0, FA-1 = 1)
    neuron_type = 0
    # Neuron response (Spiking = 0, Bursting = 1)
    neuron_response = 0

    topography_size = 100
    # topography_array = create_ring_array(topography_size)
    # topography_array = create_topography_array()
    topography_array = np.loadtxt('Topography_Data/topography_array_output_with_commas.csv', delimiter=',')

    spike_counts_array = np.zeros((topography_size, topography_size), dtype=int)

    parameters = InitializeNeuron(neuron_type, neuron_response)
    dt = 0.01
    total_time = 1000

    for i in range(topography_size):
        for j in range(topography_size):
            I = np.full(100000, topography_array[i, j])
            time, v, spike_count = NeuronSimulate(parameters, dt, total_time, I)

            # Plot or save results as needed
            spike_counts_array[i, j] = spike_count / 20

    # # Create a heatmap
    # plt.imshow(spike_counts_array, cmap='viridis', origin='lower', extent=[0, topography_size, 0, topography_size])
    # plt.colorbar(label='Number of Spikes')
    # plt.title('Neuron Response Heatmap')
    # plt.xlabel('X Axis Index')
    # plt.ylabel('Y Axis Index')
    # plt.show()

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x, y = np.meshgrid(range(topography_size), range(topography_size))
    ax.plot_surface(x, y, spike_counts_array, cmap='viridis')
    z_min = 0  # Set your desired minimum value
    z_max = 20  # Set your desired maximum value
    ax.set_zlim(z_min, z_max)

    ax.set_xlabel('X Axis Index')
    ax.set_ylabel('Y Axis Index')
    ax.set_zlabel('Number of Spikes')
    ax.set_title('Neuron Response 3D Plot')
    plt.show()

    output_folder = "Output_Data"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    plt.savefig(os.path.join(output_folder, 'Static_Model_Output.png'))


if __name__ == "__main__":
    main()

In [ ]:
# ====== SCANNING MODEL ====== #
#  - This version applies the model as a strip of sensors, scanning over a surface texture

import numpy as np
import matplotlib.pyplot as plt

# ====== MODEL PARAMETERS ====== #
#   - a, b, c, and d are neuron parameters (Izhikevich model)
#   - v_th is the threshold voltage
#   - Cm is the membrane capacitance
#   - K_x is a gain factor
#   - I is the initial current
# ============================== #
def InitializeNeuron(neuron_type, neuron_response):
    if neuron_response == 0:
        return {
            'a': 0.025,
            'b': 0.2,
            'c': -65,
            'd': 6,
            'v_th': 0.03,
            'Cm': 1,
            'K_one': 0.7,
            'K_two': 3,
            'neuron_type': neuron_type
        }
    else:
        return {
            'a': 0.02,
            'b': 0.25,
            'c': -50,
            'd': 1.5,
            'v_th': 0.03,
            'Cm': 1,
            'K_one': 0.7,
            'K_two': 3,
            'neuron_type': neuron_type
        }

# ====== MODEL EQUATIONS ====== #
#   - dv and du are the differential equations for the membrane potential and recovery variable
#   - Use Euler's method to update the state of the neuron
# ============================= #
def UpdateNeuronState(parameters, v, u, t, dt, I):
    a, b, c, d, v_th, Cm, K_one, K_two, neuron_type = parameters.values()

    if neuron_type == 0:
        dv = (0.04 * v[t]**2) + (5 * v[t]) + 140 - u[t] + (K_one / Cm) * I[t]
        du = a * (b * v[t] - u[t])

        if v[t] > v_th:
            v[t] = c
            u[t] += d

        u[t+1] = u[t] + dt * du
        v[t+1] = v[t] + dt * dv

    else:
        dv = (0.04 * v[t]**2) + (5 * v[t]) + 140 - u[t]
        du = a * (b * v[t] - u[t])

        if v[t] > v_th:
            v[t] = c
            u[t] += d


        u[t+1] = u[t] + dt * du
        v[t+1] = v[t] + dt * dv + (K_two / Cm) * (I[t + 1] - I[t])

# ====== PLOT RESULTS ====== #
#   - Membrane Potential vs Time plots the membrane potential over 1000 ms
#   - Add plot for recovery variable
#   - Add plot for firing rate vs current
# ========================== #
def PlotNeuronResults(v, spike_counts_array, total_time, topography_size):
    output_folder = "Output_Data"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    plt.xlabel('Time')
    plt.ylabel('Neuron Output')
    plt.title('Neuron Simulation Results for Each Row')
    plt.show()

    # ====== Create a heatmap ====== #
    plt.imshow(spike_counts_array[:, :len(v)], cmap='viridis', aspect='auto', origin='lower', extent=[0, total_time, 0, topography_size])
    plt.colorbar(label='Neuron Output')
    plt.title('Mechanoreceptor Response Heatmap')
    plt.xticks([])
    plt.yticks([])
    plt.xlabel('Topography Surface Axis')
    plt.ylabel('Topography Surface Axis')
    plt.savefig(os.path.join(output_folder, 'Scanning_Model_Heatmap.png'))
    plt.show()

    # ====== Create a 3D plot of the topography encoding ====== #
    fig = plt.figure(figsize=(10, 12))
    ax = fig.add_subplot(111, projection='3d')
    scaled_down_spike_counts_array = spike_counts_array[:, :len(v)] / 10.0
    x, y = np.meshgrid(np.linspace(0, total_time, len(v)), range(topography_size))
    ax.plot_surface(x, y, scaled_down_spike_counts_array, cmap='viridis')

    z_min = 0 # Set your desired minimum value
    z_max = -100  # Set your desired maximum value
    ax.set_zlim(z_min, z_max)

    plt.xticks([])
    plt.yticks([])
    ax.set_xlabel('Topography Surface Axis')
    ax.set_ylabel('Topography Surface Axis')
    ax.set_zlabel('Neuron Output')
    ax.set_title('Topography Mechanoreceptor Response 3D Plot')
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'Scanning_Model_3D_Plot.png'))
    plt.show()

# ====== SIMULATE NEURON ====== #
#   - Given the type of neuron and parameters, simulate the neuron over 1000 ms
#   - Initialize the membrane potential and recovery variable at t = 0
#   - Return the time and membrane potential
# ============================ #
def NeuronSimulate(parameters, dt, total_time, I):
    num_steps = int(total_time / dt)

    v = np.zeros(num_steps)
    u = np.zeros(num_steps)
    time = np.linspace(0, total_time, num_steps)

    v[0] = -65
    u[0] = 0

    for t in range(num_steps - 1):
        UpdateNeuronState(parameters, v, u, t, dt, I)

    return time, v

# ====== MAIN ====== #
#   - Contains all functions of the model
#   - Initialize the neuron type (Spiking = 0, Bursting = 1)
#   - Simulate the neuron
#   - Plot the results
# ================== #
def main():
    # Neuron type (SA-1 = 0, FA-1 = 1)
    neuron_type = 0
    # Neuron response (Spiking = 0, Bursting = 1)
    neuron_response = 0

    topography_size = 100

    receptor_resolution = 100000

    topography_array = np.loadtxt('Topography_Data/topography_array_output_with_commas.csv', delimiter=',')

    spike_counts_array = np.zeros((topography_size, receptor_resolution), dtype=int)

    parameters = InitializeNeuron(neuron_type, neuron_response)
    dt = 0.01
    total_time = 1000

    for i in range(topography_size):
            selected_row = topography_array[i, :]
            I = np.repeat(selected_row, 1000)
            time, v = NeuronSimulate(parameters, dt, total_time, I)

            spike_counts_array[i, :len(v)] = v

            plt.plot(v, label=f'Row {i + 1}')

    PlotNeuronResults(v, spike_counts_array, total_time, topography_size)


if __name__ == "__main__":
    main()